In [1]:
import mwapi
import datetime as dt
from requests_oauthlib import OAuth1
import secrets.oauth as oauth_cfg
import re
import time
import html
import IPython.display as ipyd

# Configuration

In [2]:
# A descriptive user agent for our API requests
USER_AGENT = (
    "CE Insights survey bot -- " +
    "https://github.com/wikimedia-research/Community-Engagement-Insights-sampling"
)

# A MassMessage list for basic testing. 
TEST_LIST = "User:Neil P. Quinn-WMF/MassMessage test list"

# Number of seconds to wait before sending each individual MassMessage
WAITING_PERIOD = 15

In [3]:
MESSAGE_PAGE = "Community Engagement Insights/MassMessages/first reminder"

# Languages other than English with a translation available
MESSAGE_LANGS = ["ar", "de", "es", "fr", "it", "ja", "nl", "pl", "pt", "ru", "uk", "zh"]

# The name of the tag used to mark the content to send on the message page
MESSAGE_MARKER = "ce-insights-content"

# Load strata

In [4]:
real_strata = pd.read_csv("data/interim/strata.tsv", sep = "\t")
real_strata.head()

,proj_group,edit_bin,sample_size,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)",26,"""[[\""Simo abchir\"", \""ar.wikipedia.org\""], [\""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
1,arwiki,"[30, 150)",94,"""[[\""Ostazwalid\"", \""ar.wikipedia.org\""], [\""M...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
2,arwiki,"[150, 600)",95,"""[[\""\\u0623\\u0628\\u0631\\u0627\\u0647\\u064...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
3,arwiki,"[600, 1200)",56,"""[[\""Assamarraie\"", \""ar.wikipedia.org\""], [\""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...
4,arwiki,"[1200, 3500)",67,"""[[\""\\u0635\\u0641\\u0627\\u0621\"", \""ar.wiki...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...


In [5]:
test_strata = pd.read_csv("data/interim/test-strata.tsv", sep = "\t")
test_strata.head()

,proj_group,edit_bin,sample_size,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)",2,"[[\Neil P. Quinn-WMF\"", \""ar.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1
1,asia_wps,"[30, 150)",2,"[[\Neil P. Quinn-WMF\"", \""ko.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2
2,cee_wps,"[150, 600)",2,"[[\Neil P. Quinn-WMF\"", \""uk.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3
3,commons,"[600, 1200)",2,"[[\Neil P. Quinn-WMF\"", \""commons.wikimedia.or...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4
4,dewiki,"[1200, 3500)",2,"[[\Neil P. Quinn-WMF\"", \""de.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5


## Add languages to strata

In [6]:
proj_langs = pd.read_csv("data/raw/project-group-languages.tsv", sep = "\t")
proj_langs.head()

,proj_group,lang
0,arwiki,ar
1,asia_wps,en
2,cee_wps,en
3,commons,en
4,dewiki,de


In [7]:
test_strata = test_strata.merge(proj_langs, how = "left", on = "proj_group")
test_strata.sample(n = 10)

,proj_group,edit_bin,sample_size,sampled_users,survey_url,preview_url,page_title,lang
14,ruwiki,"[150, 600)",2,"[[\Neil P. Quinn-WMF\"", \""ru.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ru3,ru
15,weur_wps,"[600, 1200)",2,"[[\Neil P. Quinn-WMF\"", \""sv.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/we4,en
7,frwiki,"[30, 150)",2,"[[\Neil P. Quinn-WMF\"", \""fr.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/fr2,fr
6,eswiki,"[10, 30)",2,"[[\Neil P. Quinn-WMF\"", \""es.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/es1,es
11,nlwiki,"[3500, 1100000)",2,"[[\Neil P. Quinn-WMF\"", \""nl.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/nl6,nl
10,meaf_wps,"[1200, 3500)",2,"[[\Neil P. Quinn-WMF\"", \""he.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/me5,en
13,ptwiki,"[30, 150)",2,"[[\Neil P. Quinn-WMF\"", \""pt.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/pt2,pt
9,jawiki,"[600, 1200)",2,"[[\Neil P. Quinn-WMF\"", \""ja.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ja4,ja
4,dewiki,"[1200, 3500)",2,"[[\Neil P. Quinn-WMF\"", \""de.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5,de
17,zhwiki,"[3500, 1100000)",2,"[[\Neil P. Quinn-WMF\"", \""zh.wikipedia.org\""],...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/zh6,zh


In [8]:
real_strata = real_strata.merge(proj_langs, how = "left", on = "proj_group")
real_strata.sample(n = 10)

,proj_group,edit_bin,sample_size,sampled_users,survey_url,preview_url,page_title,lang
22,commons,"[1200, 3500)",87,"""[[\""Druyts.t\"", \""commons.wikimedia.org\""], [...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,en
42,frwiki,"[10, 30)",65,"""[[\""Nuitsfauves\"", \""fr.wikipedia.org\""], [\""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,fr
58,jawiki,"[1200, 3500)",44,"""[[\""\\u30b8\\u30e3\\u30e0\\u30ea\\u30f3\"", \""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,ja
31,enwiki,"[30, 150)",143,"""[[\""Shivanshu.gupta.mec16\"", \""en.wikipedia.o...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,en
95,weur_wps,"[3500, 1100000)",324,"""[[\""Danmichaelo\"", \""no.wikipedia.org\""], [\""...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,en
45,frwiki,"[600, 1200)",137,"""[[\""Ludo29\"", \""fr.wikipedia.org\""], [\""Abbd-...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,fr
37,eswiki,"[30, 150)",127,"""[[\""Mart\\u00ednTantale\\u00e1n\"", \""es.wikip...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,es
2,arwiki,"[150, 600)",95,"""[[\""\\u0623\\u0628\\u0631\\u0627\\u0647\\u064...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,ar
99,wikidata,"[600, 1200)",35,"""[[\""Fililipa\"", \""www.wikidata.org\""], [\""And...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,en
77,other,"[3500, 1100000)",277,"""[[\""Otourly\"", \""fr.wiktionary.org\""], [\""Car...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,Community Engagement Insights/MassMessages/Lis...,en


# Set up connection

Use OAuth to authenticate as an [owner-only consumer](https://www.mediawiki.org/wiki/OAuth/Owner-only_consumers).

In [9]:
sess = mwapi.Session("https://meta.wikimedia.org", user_agent = USER_AGENT)

auth = OAuth1(
    oauth_cfg.consumer_token,
    oauth_cfg.consumer_secret,
    oauth_cfg.access_token,
    oauth_cfg.access_secret
)

def get_token():
    resp = sess.get(
        action="query", 
        meta="tokens", 
        type="csrf", 
        auth = auth
    )
    
    return resp["query"]["tokens"]["csrftoken"]

def api_get(*args, **kwargs):
    return sess.get(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        **kwargs
    )
    
def api_post(*args, **kwargs):
    return sess.post(
        *args,
        format = "json",
        formatversion = 2,
        auth = auth,
        token = get_token(),
        **kwargs
    )

# Check API authentication 

In [10]:
api_get(
    action = "query",
    meta = "userinfo"
)["query"]["userinfo"]["name"]

'WMF Surveys'

# Assemble message

This uses the ["Assemble multilingual message" Lua module](https://meta.wikimedia.org/wiki/Module:Assemble_multilingual_message), 
as described at [meta:Newsletters/Translation#Regular process](https://meta.wikimedia.org/wiki/Newsletters/Translation#Regular_process).

In [11]:
# Be careful with the arrangement of linebreaks and pipes between the languages.
# In some cases, this can cause the leading or trailing language not to be included in the output.
message_assembly = """
{{{{#invoke:
Assemble multilingual message|
assembleMessage|
marker={marker}|
page={page}|{langs}
}}}}
""".format(
    page = MESSAGE_PAGE,
    marker = MESSAGE_MARKER,
    langs = "|".join(MESSAGE_LANGS) + "|"
)

print(message_assembly)


{{#invoke:
Assemble multilingual message|
assembleMessage|
marker=ce-insights-content|
page=Community Engagement Insights/MassMessages/first reminder|ar|de|es|fr|it|ja|nl|pl|pt|ru|uk|zh|
}}



In [12]:
raw_msg = api_get(
    action = "parse",
    text = message_assembly,
    prop = "text",
    contentmodel = "wikitext",
    disablelimitreport = True
)["parse"]["text"]

In [13]:
# Strip ALL instances of `<pre>` and `</pre>` from the message
# Replace HTML entities (mainly angle brackets) with raw characters
def strip_pre(msg):
    return re.sub(
        r"<[/]?pre>",
        "",
        msg
    )

# Convert the date-only signature to a date-and-name signature
def convert_sig(msg):
    return re.sub(
        r"~~~~~",
        """<span class="mw-content-ltr" dir="ltr">[[m:User:WMF Surveys|WMF Surveys]]</span>, ~~~~~""",
        msg
    )

unlinked_msg = html.unescape(strip_pre(convert_sig(raw_msg)))

In [14]:
print(unlinked_msg)

<div class="mw-parser-output">
{{subst:#switch:{{subst:CONTENTLANG}}|it=<div class="plainlinks mw-content-ltr" lang="it" dir="ltr">
Ogni risposta in questo sondaggio aiuta la Fondazione Wikimedia a migliorare la tua esperienza sui progetti Wikimedia. Finora, abbiamo ricevuto commenti dal 29% degli utenti Wikimedia. Il sondaggio è disponibile in varie lingue e richiede tra i 20 e i 40 minuti per essere completato. '''[https://www.example.com  Inizia il sondaggio adesso.]'''

Se hai già compilato il sondaggio, siamo spiacenti che tu abbia ricevuto questo avviso. Abbiamo progettato il sondaggio in modo che sia impossibile identificare gli utenti che l'hanno già compilato, quindi dobbiamo inviare un avviso a tutti.
<span class="mw-translate-fuzzy">Se non vuoi ricevere ulteriori avvisi o non vuoi essere più informato sui futuri sondaggi, invia una mail tramite la funzione InviaEmail a [[:m:Special:EmailUser/WMF Surveys|WMF Surveys]]</span>.  Puoi anche inviare qualsiasi domanda tu abbia a q

In [15]:
# Replace links
def add_link(msg, link):
    return re.sub(
        r"https://www.example.com",
        link,
        msg
    )

## Grab headings

In [16]:
def get_first_heading(lang):
    # This assumes the section heading we want is the first one on the page
    return api_get(
        action = "parse",
        page = MESSAGE_PAGE + "/" + lang,
        prop = "sections"
    )["parse"]["sections"][0]["line"]

In [17]:
titles = {}

titles["en"] = get_first_heading("en")

for lang in MESSAGE_LANGS:
    titles[lang] = get_first_heading(lang)
    
titles

{'ar': 'تذكير: يرجى مشاركة آرائك وملاحظاتك في استبيان ويكيميديا هذا',
 'de': 'Erinnerung: Teile dein Feedback in dieser Wikimedia-Umfrage',
 'en': 'Reminder: Share your feedback in this Wikimedia survey',
 'es': 'Recordatorio: Comparta sus comentarios en esta encuesta de Wikimedia',
 'fr': 'Rappel\xa0: partagez vos commentaires dans ce sondage Wikimédia',
 'it': 'Messaggio: Condividi i tuoi commenti in questo sondaggio Wikimedia',
 'ja': 'お知らせ: このウィキメディア・アンケートでご意見を聞かせてください',
 'nl': 'Herinnering: Deel je feedback in de Wikimedia-enquête',
 'pl': 'Przypomnienie: Wyraź swoją opinię w badaniu Wikimedia',
 'pt': 'Lembrete: compartilhe seus comentários nesta pesquisa da Wikimedia',
 'ru': 'Напоминание: Поделитесь своим мнением в этом опросе Викимедиа',
 'uk': 'Нагадування: Поділіться своїми відгуками в цьому опитуванні Вікімедіа',
 'zh': '提醒：通过此次维基媒体调查分享您的反馈意见'}

# Set up messaging

In [18]:
def send_message(target_list, subj, msg):
    api_post(
        action = "massmessage",
        spamlist = target_list,
        subject = subj,
        message = msg
    )


def message_frame(frame):
    for row in frame.itertuples():
        if row.sample_size > 0:
            message_list = row.page_title
            survey_url = row.survey_url
            preview_url = row.preview_url
            title_lang = row.lang

            linked_msg = add_link(unlinked_msg, survey_url)

            print_err(
                "Sending to {message_list} with url {survey_url} in {wait} seconds".format(
                    message_list = message_list,
                    survey_url = survey_url,
                    wait = WAITING_PERIOD
                )
            )

            time.sleep(WAITING_PERIOD)

            send_message(message_list, titles[title_lang], linked_msg)

            print_err("Sent.")

# Test

## Basic test

In [ ]:
print(unlinked_msg)

In [ ]:
send_message(
    TEST_LIST,
    "Testing MassMessage",
    unlinked_msg
)

## Test with full message

In [19]:
message_frame(test_strata)

Sending to User:Neil P. Quinn/2018 CE Insights/ar1 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=1&prjedc=ar1 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/as2 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=as&edc=2&prjedc=as2 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/ce3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ce&edc=3&prjedc=ce3 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/co4 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=co&edc=4&prjedc=co4 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/de5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=de&edc=5&prjedc=de5 in 15 seconds
Sent.
Sending to User:Neil P. Quinn/2018 CE Insights/en6 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=en&edc=6&pr

# ACTUAL SENDING

In [20]:
# Run this cell and follow the prompt to unlock sending code for 30 seconds
ipyd.HTML("""
<script type="text/Javascript">
var kernel = IPython.notebook.kernel;

if (prompt("The code below will send talk pages messages to thousands of editors. Type 'run' to unlock it")) {
    kernel.execute("allow_run = dt.datetime.now()")
};
</script
""")

In [21]:
# If the cell above has been run within the past 30 seconds, send the messages
if (dt.datetime.now() - allow_run) < dt.timedelta(seconds = 30):
    message_frame(real_strata)
else:
    print_err(
        "This code will send talk page messages to thousands of editors.", 
        "If you're certain you want to do this, run the cell above to unlock it.",
        sep = "\n"
    )

Sending to Community Engagement Insights/MassMessages/Lists/2018/ar1 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=1&prjedc=ar1 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ar2 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=2&prjedc=ar2 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ar3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=3&prjedc=ar3 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ar4 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=ar&edc=4&prjedc=ar4 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ar5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=ar&edc=5&prjedc=ar5 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages

Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/fr3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=fr&edc=3&prjedc=fr3 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/fr4 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=fr&edc=4&prjedc=fr4 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/fr5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=fr&edc=5&prjedc=fr5 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/fr6 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=fr&edc=6&prjedc=fr6 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/it1 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=it&edc=1&prjedc=it1 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassM

Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ru5 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=ru&edc=5&prjedc=ru5 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/ru6 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=ru&edc=6&prjedc=ru6 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/we1 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=we&edc=1&prjedc=we1 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/we2 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=we&edc=2&prjedc=we2 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMessages/Lists/2018/we3 with url https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=we&edc=3&prjedc=we3 in 15 seconds
Sent.
Sending to Community Engagement Insights/MassMe